In [2]:
from Agents.RandomAgent import RandomAgent as ra
from Agents.AdrianHerasAgent import AdrianHerasAgent as aha
from Agents.AlexPastorAgent import AlexPastorAgent as apa
from Agents.AlexPelochoJaimeAgent import AlexPelochoJaimeAgent as apja
from Agents.CarlesZaidaAgent import CarlesZaidaAgent as cza
from Agents.CrabisaAgent import CrabisaAgent as ca
from Agents.EdoAgent import EdoAgent as ea
from Agents.PabloAleixAlexAgent import PabloAleixAlexAgent as paaa
from Agents.SigmaAgent import SigmaAgent as sa
from Agents.TristanAgent import TristanAgent as ta

from Managers.GameDirector import GameDirector
AGENTS = [ra, aha, apa]
# AGENTS = [ra, aha, apa, apja, cza, ca, ea, paaa, sa, ta]


In [2]:
#Importing dependencies for DEAP
import random
import numpy as np
from deap import base, creator, tools

In [3]:
# -------------------------------------------------------------------
# 1. Definir Estructuras de Fitness e Individuo
# -------------------------------------------------------------------
# Suponiendo que queremos maximizar la función de evaluación:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [4]:
# -------------------------------------------------------------------
# 2. Inicialización del Individuo (lista de 10 probabilidades que suman 1)
# -------------------------------------------------------------------
def initIndividual(icls, size):
    # Genera 10 números aleatorios y normaliza para que sumen 1
    vec = [random.random() for _ in range(size)]
    s = sum(vec)
    individual = [x / s for x in vec]
    return icls(individual)

toolbox = base.Toolbox()
toolbox.register("individual", initIndividual, creator.Individual, size=10)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [5]:
# -------------------------------------------------------------------
# 3. Función de Simulación (Ejemplo ilustrativo)
# -------------------------------------------------------------------
def simular_catan(probabilidades):
    """
    Función simuladora de Catan.
    Utiliza el vector de probabilidades para determinar la probabilidad
    de elegir a cada agente en la partida.
    
    Aquí deberías integrar tu simulador real. En este ejemplo,
    retornamos un valor aleatorio que simula el desempeño.
    """
    # Ejemplo: se simula el rendimiento basado en la distribución
    rendimiento = sum(probabilidades) * random.uniform(0.5, 1.5)
    return rendimiento

In [6]:
# -------------------------------------------------------------------
# 4. Función de Evaluación
# -------------------------------------------------------------------
def evaluate(individual):
    """
    Evalúa un individuo simulando varias partidas.
    Se selecciona un agente en cada partida de forma probabilística
    usando el vector 'individual'.
    """
    num_partidas = 20  # Puedes ajustar el número de simulaciones
    resultados = []
    for _ in range(num_partidas):
        resultado = simular_catan(individual)
        resultados.append(resultado)
    # La aptitud es el promedio de resultados obtenidos
    promedio = np.mean(resultados)
    return promedio,  # Nota: siempre se retorna una tupla

toolbox.register("evaluate", evaluate)

In [7]:
# -------------------------------------------------------------------
# 5. Operadores Genéticos con Normalización
# -------------------------------------------------------------------
# a) Cruce: Usamos cxBlend y luego normalizamos para que la suma sea 1
def cxBlendNormalize(ind1, ind2, alpha=0.5):
    tools.cxBlend(ind1, ind2, alpha)
    s1, s2 = sum(ind1), sum(ind2)
    for i in range(len(ind1)):
        ind1[i] /= s1
        ind2[i] /= s2
    return ind1, ind2

toolbox.register("mate", cxBlendNormalize)

# b) Mutación: Aplicamos mutGaussian y normalizamos el individuo
def mutGaussianNormalize(individual, mu, sigma, indpb):
    tools.mutGaussian(individual, mu, sigma, indpb)
    s = sum(individual)
    # Asegurarse de no dividir por cero y evitar valores negativos
    if s != 0:
        # Si es necesario, imponemos un límite inferior a cada valor (ej. 0.001)
        individual[:] = [max(x, 0.001) for x in individual]
        s = sum(individual)
        individual[:] = [x / s for x in individual]
    return individual,

toolbox.register("mutate", mutGaussianNormalize, mu=0, sigma=0.1, indpb=0.2)

# c) Selección: Selección por torneo
toolbox.register("select", tools.selTournament, tournsize=3)


In [ ]:
# -------------------------------------------------------------------
# 6. Ciclo Evolutivo
# -------------------------------------------------------------------
random.seed(42)
pop = toolbox.population(n=50)  # Población inicial de 50 individuos
CXPB, MUTPB, NGEN = 0.5, 0.2, 40  # Probabilidades de cruce, mutación y número de generaciones

print("Inicio de la evolución")
# Evaluar la población inicial
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

print("Evaluados %i individuos" % len(pop))

# Bucle evolutivo
for g in range(NGEN):
    print("-- Generación %i --" % g)
    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    # Aplicar cruce en parejas
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # Aplicar mutación
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluar individuos con fitness no válidos
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Actualizar población
    pop[:] = offspring
    fits = [ind.fitness.values[0] for ind in pop]
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Promedio %s" % (sum(fits) / len(fits)))
    print("  Std %s" % (np.std(fits)))

best_ind = tools.selBest(pop, 1)[0]
print("Mejor individuo es %s, con fitness %s" % (best_ind, best_ind.fitness.values))

In [ ]:

def execute():
    # Ejemplo de ejecución
    try:
        game_director = GameDirector(
            agents=AGENTS, max_rounds=1, store_trace=False)

        print("PASE INSTANCIACION")
        game_trace = game_director.game_start(print_outcome=False)
    except Exception as e:
        print(f"Error: {e}")
        return 0

    # Análisis de resultados
    last_round = max(game_trace["game"].keys(),
                     key=lambda r: int(r.split("_")[-1]))
    last_turn = max(game_trace["game"][last_round].keys(
    ), key=lambda t: int(t.split("_")[-1].lstrip("P")))
    victory_points = game_trace["game"][last_round][last_turn]["end_turn"]["victory_points"]

    winner = max(victory_points, key=lambda player: int(
        victory_points[player]))
    fitness = 0
    print("WINNER")
    print(winner)
    print(int(winner.lstrip("J")))

    if AGENTS.index(winner) == int(winner.lstrip("J")):
        fitness += 1